In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('kinase_info.csv')

In [4]:
df.columns

Index(['kinase', 'ID_coral', 'uniprot', 'ID_HGNC', 'modi_group', 'group',
       'family', 'subfamily_coral', 'subfamily', 'in_pspa_st', 'in_pspa_tyr',
       'in_pspa', 'in_cddm', 'kd_ID', 'active_D1_D2', 'active_kd_ID',
       'pspa_ID', 'pseudo', 'pspa_category_small', 'pspa_category_big',
       'cddm_big', 'cddm_small', 'length', 'human_uniprot_sequence',
       'kinasecom_domain', 'nucleus', 'cytosol', 'cytoskeleton',
       'plasma membrane', 'mitochondrion', 'Golgi apparatus',
       'endoplasmic reticulum', 'vesicle', 'centrosome', 'aggresome',
       'main_location'],
      dtype='object')

In [6]:
df.pspa_category_small = df.pspa_category_small.str.capitalize()

In [7]:
df.pspa_category_big = df.pspa_category_big.str.capitalize()

In [9]:
df.to_csv('kinase_info2.csv',index=False)

In [3]:
df = pd.read_excel('uniprot_kd_labeled.xlsx')

In [5]:
df.to_parquet('uniprot_kd_labeled.parquet')

In [6]:
df = pd.read_parquet('uniprot_kd_labeled.parquet')

In [8]:
df.active_D1_D2.value_counts()

active_D1_D2
1    4209
0    1327
Name: count, dtype: int64